In [1]:
import json 
import random
import pandas as pd

In [38]:
import os
import json
import copy
import torch
import logging

from torch import nn
from datasets import load_dataset, DatasetDict, Dataset
import transformers
from peft import LoraConfig, get_peft_model
# from trl import SFTTrainer
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

In [39]:
RUS = "<RUS>"
UDM = "<UDM>"
KOMI = "<KOMI>"
MHR = "<MHR>"
EOS = "<eos>"
MANS = "<MANS>"
TRANSLATE = "<TRANSLATE>"

In [40]:
def data_articles():
    mans_data = []
    rus_data = []
    for i in range(13, 25):
        dt = pd.read_csv(f"dataMans/LUIMA-SERIPOS_20{i}_RUS_MANS.csv")
        mans = dt.mans.tolist()
        rus = dt.rus.tolist()
        for i in range(len(rus)):
            mans_data.append(MANS + mans[i].replace("\n", " ").strip() + EOS)
            rus_data.append(RUS + rus[i].replace("\n", " ").strip() + EOS)

    df_text = pd.read_csv("dataMans/READ_LITER.csv")
    mans = df_text.mans.tolist()
    rus = df_text.rus.tolist()
    for i in range(len(rus)):
        mans_data.append(MANS + mans[i] + EOS)
        rus_data.append(RUS + rus[i] + EOS)
    return mans_data, rus_data

In [41]:
def read_other(language_count=20000, count_mans=5):
    data = []
    random.seed(42)
    df_komi = pd.read_csv("dataMans/komi-rus.csv")
    index = df_komi.index.tolist()
    random.shuffle(index)
    komi = df_komi.loc[index[:language_count]]["Коми язык"].tolist()
    rus = df_komi.loc[index[:language_count]]["Русский язык"].tolist()
    for i in range(len(rus)):
        data.append(RUS + rus[i] + TRANSLATE + KOMI + komi[i] + EOS)

    komi = df_komi.loc[index[language_count:2 * language_count]]["Коми язык"].tolist()
    rus = df_komi.loc[index[language_count:2 * language_count]]["Русский язык"].tolist()
    for i in range(len(rus)):
        data.append(KOMI + komi[i] + TRANSLATE + RUS + rus[i] + EOS)
    
    df_udmurt = pd.read_csv("dataMans/udmurt-rus.csv")
    index = df_udmurt.index.tolist()
    random.shuffle(index)
    
    udm = df_udmurt.loc[index[:language_count]].udm.tolist()
    rus = df_udmurt.loc[index[:language_count]].ru.tolist()

    for i in range(len(rus)):
        data.append(RUS + rus[i] + TRANSLATE + UDM + udm[i] + EOS)

    udm = df_udmurt.loc[index[language_count:2 * language_count]].udm.tolist()
    rus = df_udmurt.loc[index[language_count:2 * language_count]].ru.tolist()

    for i in range(len(rus)):
        data.append(UDM + udm[i] + TRANSLATE + RUS + rus[i] + EOS)

    
    df_mhr = pd.read_csv("dataMans/mhr-rus.csv")
    df_mhr = df_mhr.dropna(subset=["mhr"])
    index = df_mhr.index.tolist()
    random.shuffle(index)
    
    mhr = df_mhr.loc[index[:language_count]].mhr.tolist()
    rus = df_mhr.loc[index[:language_count]].rus.tolist()

    for i in range(len(rus)):
        data.append(RUS + rus[i] + TRANSLATE + MHR + mhr[i] + EOS)

    mhr = df_mhr.loc[index[language_count:2 * language_count]].mhr.tolist()
    rus = df_mhr.loc[index[language_count:2 * language_count]].rus.tolist()

    for i in range(len(rus)):
        data.append(MHR + mhr[i] + TRANSLATE + RUS + rus[i] + EOS)

    df_text = pd.read_csv("dataMans/Gospel_Mark_RUS_MANS.csv")
    index = df_text.index.tolist()
    random.shuffle(index)
    
    mans = df_text.loc[index[:language_count]].mans.tolist()
    rus = df_text.loc[index[:language_count]].rus.tolist()

    for i in range(len(rus)):
        for j in range(count_mans):
            data.append(RUS + rus[i] + TRANSLATE + MANS + mans[i] + EOS)

    udm = df_text.loc[index[language_count:2 * language_count]].mans.tolist()
    rus = df_text.loc[index[language_count:2 * language_count]].rus.tolist()

    for i in range(len(rus)):
        for j in range(count_mans):
            data.append(MANS + mans[i] + TRANSLATE + RUS + rus[i] + EOS)

    df_text = pd.read_csv("dataMans/Book_of_John_RUS_MANS.csv")
    index = df_text.index.tolist()
    random.shuffle(index)
    
    mans = df_text.loc[index[:language_count]].mans.tolist()
    rus = df_text.loc[index[:language_count]].rus.tolist()

    for i in range(len(rus)):
        for j in range(count_mans):
            data.append(RUS + rus[i] + TRANSLATE + MANS + mans[i] + EOS)

    udm = df_text.loc[index[language_count:2 * language_count]].mans.tolist()
    rus = df_text.loc[index[language_count:2 * language_count]].rus.tolist()

    for i in range(len(rus)):
        for j in range(count_mans):
            data.append(MANS + mans[i] + TRANSLATE + RUS + rus[i] + EOS)

    df_text = pd.read_csv("dataMans/Bible_UDM_RUS.csv")
    df_text = df_text.dropna(subset=["udm"])
    df_text = df_text.dropna(subset=["rus"])
    index = df_text.index.tolist()
    random.shuffle(index)
    
    udm = df_text.loc[index[:language_count]].udm.tolist()
    rus = df_text.loc[index[:language_count]].rus.tolist()

    for i in range(len(rus)):
        data.append(RUS + rus[i] + TRANSLATE + UDM + udm[i] + EOS)

    udm = df_text.loc[index[language_count:2 * language_count]].udm.tolist()
    rus = df_text.loc[index[language_count:2 * language_count]].rus.tolist()

    for i in range(len(rus)):
        data.append(UDM + udm[i] + TRANSLATE + RUS + rus[i] + EOS)
    random.shuffle(data)
    return data

In [42]:
translate_examples = read_other()
articles_mans, articles_rus = data_articles()

In [43]:
data = []
data.extend(translate_examples)
data.extend(articles_rus)
for elem in articles_mans:
    data.append(elem)
    data.append(elem)
    data.append(elem)
random.seed(7)
random.shuffle(data)

In [44]:
len(data)

168822

In [25]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
with open("../train/spm_man.vocab", "r+") as file1:
    vocab = [elem.split("\t")[0] for elem in file1.read().split("\n")]

RUS = "<RUS>"
UDM = "<UDM>"
KOMI = "<KOMI>"
MHR = "<MHR>"
MANS = "<MANS>"
TRANSLATE = "<TRANSLATE>"
COMPUTE = "<COMPUTE>"
special_tokens_dict = {"additional_special_tokens": [RUS, UDM, KOMI, MHR, MANS]}
# vocab.extend([RUS, UDM, KOMI, MHR, MANS])
new_tokens = set(vocab) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
tokenizer.add_special_tokens(special_tokens_dict)

5

In [48]:
data[3]

'<RUS>А на заводе железобетонных изделий «ЗЖБИ Арктика» в городе предусмотрено создать линию для выпуска железобетонной продукции с учетом потребности рынка на северных, арктических территориях Коми и соседних регионов.<TRANSLATE><KOMI>А карса «ЗЖБИ Арктика» кӧрт бетона изделиеяс вӧчан заводын кӧсйӧны лэдзны кӧрт бетона продукция Комилӧн да орчча регионъяслӧн войвыв, арктика мутасъясын коланлун тӧд вылын кутӧмӧн.<eos>'

In [29]:
sent = data[1][:100]
print(sent)
tokenizer.encode(sent, add_special_tokens=False)

<MHR>Ой, Альбина ӱдырем толешыс.<RUS>Ой, Альбина, доченька, пришла.<eos>


[270792,
 225199,
 235269,
 95895,
 74667,
 235248,
 244672,
 267554,
 4550,
 3572,
 1480,
 235426,
 267517,
 235265,
 270789,
 225199,
 235269,
 95895,
 74667,
 235269,
 2879,
 14312,
 1176,
 235269,
 187709,
 235265,
 1]

In [11]:
tokenizer.encode(data[0], add_special_tokens=False)

[235248,
 270789,
 16339,
 11147,
 8211,
 268685,
 45150,
 975,
 260668,
 235328,
 235269,
 2087,
 11024,
 235344,
 269846,
 235326,
 10292,
 1416,
 1950,
 269693,
 262282,
 256483,
 235326,
 235269,
 18880,
 2087,
 78708,
 26502,
 262125,
 2879,
 174738,
 235248,
 264019,
 2886,
 235350,
 78387,
 235269,
 18880,
 2087,
 33629,
 235402,
 256735,
 20668,
 2879,
 174738,
 21377,
 1215,
 40498,
 4501,
 235265,
 235248,
 270790,
 260668,
 93526,
 138523,
 264752,
 235269,
 256483,
 258430,
 261697,
 138523,
 258710,
 1157,
 108,
 267548,
 2886,
 235350,
 263893,
 264794,
 261831,
 235269,
 108,
 10258,
 265170,
 264756,
 263026,
 265824,
 269420,
 235320,
 264794,
 261741,
 235265,
 1]

In [49]:
dt = {}
dt['train'] = data[:int(len(data) * 0.98)]
dt['test'] = data[int(len(data) * 0.98):]

In [50]:
out_file = open("data.json", "w")  
    
json.dump(dt, out_file, indent = 6)  
    
out_file.close()

In [37]:
len(dt['train']), len(dt['test'])

(165445, 3377)